이 튜토리얼은 "밑바닥부터 시작하는 데이터과학" 의 gibbs sampling 과 LDA 구현하기의 코드입니다. 코드에 대한 해석은 해당 책 혹은 [ratsgo 의 블로그 포스트][ratsgo]를 참고 하시기 바랍니다.

[ratsgo]: https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/07/09/lda/

## gibbs sampling

    x: 주사위 1 의 랜덤 샘플
    y: 주사위 1 과 2 의 랜덤 샘플의 합

In [1]:
import random
from collections import defaultdict

def roll_a_die():
    return random.choice([1,2,3,4,5,6])

def direct_sample():
    d1 = roll_a_die()
    d2 = roll_a_die()
    return d1, d1 + d2

def random_y_given_x(x):
    return x + roll_a_die()

def random_x_given_y(y):
    if y <= 7:
        return random.randrange(1, y)
    else:
        return random.randrange(y - 6, 7)

def gibbs_sample(num_iter=100):
    # initialization
    # dosent matter which value x and y have.
    x, y = 1, 2
    for _ in range(num_iter):
        x = random_x_given_y(y)
        y = random_y_given_x(x)
    return x, y

def compare_distributions(num_samples=1000):
    counts = defaultdict(lambda: [0, 0])
    for _ in range(num_samples):
        counts[gibbs_sample()][0] += 1
        counts[direct_sample()][1] += 1
    return dict(counts)

In [2]:
compare_distributions()

{(1, 2): [27, 32],
 (1, 3): [30, 32],
 (1, 4): [28, 22],
 (1, 5): [28, 29],
 (1, 6): [30, 30],
 (1, 7): [13, 24],
 (2, 3): [27, 23],
 (2, 4): [33, 23],
 (2, 5): [31, 28],
 (2, 6): [36, 25],
 (2, 7): [36, 32],
 (2, 8): [22, 29],
 (3, 4): [24, 20],
 (3, 5): [26, 26],
 (3, 6): [27, 27],
 (3, 7): [25, 30],
 (3, 8): [28, 22],
 (3, 9): [33, 33],
 (4, 5): [27, 32],
 (4, 6): [32, 31],
 (4, 7): [31, 31],
 (4, 8): [28, 29],
 (4, 9): [23, 30],
 (4, 10): [22, 29],
 (5, 6): [22, 22],
 (5, 7): [30, 25],
 (5, 8): [19, 37],
 (5, 9): [31, 34],
 (5, 10): [25, 25],
 (5, 11): [32, 24],
 (6, 7): [25, 22],
 (6, 8): [35, 28],
 (6, 9): [28, 34],
 (6, 10): [32, 25],
 (6, 11): [20, 31],
 (6, 12): [34, 24]}

## lda

In [3]:
def sample_from(weights):
    """a sample from weights using cumulative prob. function"""
    total = sum(weights)
    rnd = total * random.random()
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0:
            return i

In [4]:
documents = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

In [5]:
from collections import Counter

K = 4 # num of topics

document_topic_counts = [Counter() for _ in documents]
topic_word_counts = [Counter() for _ in range(K)]
topic_counts = [0 for _ in range(K)]
document_lengths = list(map(len, documents))
distinct_words = set(word for document in documents for word in document)

W = len(distinct_words)
D = len(documents)

In [6]:
def p_topic_given_document(topic, d, alpha=0.1):
    return ((document_topic_counts[d][topic] + alpha) / 
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + W * beta))

def topic_weight(d, word, k):
    """weight of topic k given a word and a doc"""
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

def w_topic_given_word(word, as_str=True):
    weights = [p_word_given_topic(word, k) for k in range(K)]
    if as_str:
        weights = ['%.4f' % w for w in weights]
    return weights

In [7]:
# initialize
n_iter = 1000
seed = 0

random.seed(seed)

document_topics = [[random.randrange(K) for word in document]
                    for document in documents]

for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

for i_iter in range(n_iter):

    if i_iter % 100 == 0:
        print(w_topic_given_word('statistics'))

    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d], document_topics[d])):
            
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1
            
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic
            
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

['0.0561', '0.0054', '0.0466', '0.0561']
['0.0051', '0.0079', '0.0509', '0.0761']
['0.0509', '0.0041', '0.0060', '0.1129']
['0.1084', '0.0039', '0.0094', '0.0060']
['0.1505', '0.0046', '0.0051', '0.0051']
['0.1047', '0.0057', '0.0057', '0.0060']
['0.1047', '0.0044', '0.0060', '0.0079']
['0.1084', '0.0051', '0.0049', '0.0079']
['0.1260', '0.0068', '0.0046', '0.0049']
['0.1047', '0.0051', '0.0074', '0.0054']


In [8]:
topic_word_counts[2].most_common()

[('regression', 3),
 ('Python', 2),
 ('R', 2),
 ('libsvm', 2),
 ('scikit-learn', 2),
 ('mathematics', 1),
 ('support vector machines', 1),
 ('Haskell', 1),
 ('Mahout', 1),
 ('Java', 0),
 ('Cassandra', 0),
 ('MongoDB', 0),
 ('Postgres', 0),
 ('scipy', 0),
 ('statsmodels', 0),
 ('probability', 0),
 ('machine learning', 0),
 ('statistics', 0),
 ('C++', 0),
 ('artificial intelligence', 0),
 ('HBase', 0),
 ('NoSQL', 0),
 ('numpy', 0),
 ('theory', 0),
 ('Hadoop', 0),
 ('Spark', 0),
 ('Storm', 0),
 ('pandas', 0),
 ('programming languages', 0),
 ('neural networks', 0),
 ('deep learning', 0),
 ('decision trees', 0),
 ('Big Data', 0),
 ('MapReduce', 0),
 ('databases', 0),
 ('MySQL', 0)]

In [9]:
for k, word_counts in enumerate(topic_word_counts):
    for word, count in word_counts.most_common():
        if count > 0:
            print(k, word, count)

0 Java 3
0 Big Data 3
0 Hadoop 2
0 HBase 1
0 C++ 1
0 Spark 1
0 Storm 1
0 programming languages 1
0 MapReduce 1
0 Cassandra 1
0 deep learning 1
1 HBase 2
1 neural networks 2
1 Postgres 2
1 MongoDB 2
1 machine learning 2
1 Cassandra 1
1 numpy 1
1 decision trees 1
1 deep learning 1
1 databases 1
1 MySQL 1
1 NoSQL 1
1 artificial intelligence 1
1 scipy 1
2 regression 3
2 Python 2
2 R 2
2 libsvm 2
2 scikit-learn 2
2 mathematics 1
2 support vector machines 1
2 Haskell 1
2 Mahout 1
3 statistics 3
3 probability 3
3 Python 2
3 R 2
3 pandas 2
3 statsmodels 2
3 C++ 1
3 artificial intelligence 1
3 theory 1
